**Imports**

In [1]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Resizing, Input
import numpy as np
from tensorflow.keras.applications import VGG16
import tensorflow as tf

# 1. Cargar el dataset Fashion-MNIST

In [2]:
# Cargar datos
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalizar (0-255 → 0-1)
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Convertir etiquetas a one-hot
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# 2. Construcción y entrenamiento de los diferentes modelos
## Modelo 1: Dos bloques convolucionales + capa densa final (CNN sencilla)

### Construcción del modelo

In [12]:
model1 = Sequential()

# Bloque 1
model1.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(28, 28, 1)))
model1.add(MaxPooling2D(pool_size=(2,2)))

# Bloque 2
model1.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model1.add(MaxPooling2D(pool_size=(2,2)))

# Capa final densa
model1.add(Flatten())
model1.add(Dense(10, activation='softmax'))

model1.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │        31,370 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,186 (196.04 KB)

 Trainable params: 50,186 (196.04 KB)

 Non-trainable params: 0 (0.00 B)

### Entrenamiento

In [13]:
model1.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

model1.fit(x_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7613 - loss: 0.6726
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8846 - loss: 0.3286
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9002 - loss: 0.2830
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9101 - loss: 0.2532
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9186 - loss: 0.2282
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9234 - loss: 0.2146
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9288 - loss: 0.1999
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9360 - loss: 0.1797
Epoch 9/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9387 - loss: 0.1690
Epoch 10/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9431 - loss: 0.1562


## Modelo 2: CNN más profunda

In [17]:
model2 = Sequential()

# Bloque 1
model2.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(28,28,1)))
model2.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2,2)))
model2.add(Dropout(0.25))

# Bloque 2
model2.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model2.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2,2)))
model2.add(Dropout(0.25))

# Bloque 3
model2.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2,2)))
model2.add(Dropout(0.25))

# Parte densa
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(10, activation='softmax'))

model2.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 436,586 (1.67 MB)

 Trainable params: 436,586 (1.67 MB)

 Non-trainable params: 0 (0.00 B)

### Entrenamiento

In [18]:
model2.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

model2.fit(x_train, y_train, epochs=20, batch_size=64)

Epoch 1/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.6838 - loss: 0.8491
Epoch 2/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8627 - loss: 0.3673
Epoch 3/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8889 - loss: 0.3028
Epoch 4/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8958 - loss: 0.2800
Epoch 5/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9037 - loss: 0.2581
Epoch 6/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9126 - loss: 0.2380
Epoch 7/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9160 - loss: 0.2299
Epoch 8/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9186 - loss: 0.2187
Epoch 9/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9207 - loss: 0.2136
Epoch 10/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9225 - loss: 0.2095
Epoch 11/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9274 - loss: 0.1978
Epoch 12/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/st

## Modelo 3: VGG16 sin entrenamiento previo (desde cero)

In [7]:
# Instanciamos la VGG16 base
# Le decimos que espere 96x96x3, porque nuestras capas previas se lo darán así.
vgg_base_scratch = VGG16(
    weights=None,          # sin pesos
    include_top=False,
    input_shape=(96, 96, 3)
)

model3 = Sequential()

# --- BLOQUE DE ADAPTACIÓN (para ahorrar RAM) ---
# 1. Entrada real de tus datos (pequeña)
model3.add(Input(shape=(28, 28, 1)))

# 2. El modelo se encarga de agrandar la imagen a 96x96
model3.add(Resizing(96, 96))

# 3. Convertir de 1 canal (Grayscale) a 3 canales (RGB)
# Usamos una convolución de 1x1 con 3 filtros. Es más eficiente que repetir y el modelo "aprende" la mejor forma de
# colorear la imagen.
model3.add(Conv2D(3, (1, 1), padding='same'))

model3.add(vgg_base_scratch)
model3.add(Flatten())
model3.add(Dense(256, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(10, activation='softmax'))

model3.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing (Resizing)             │ (None, 96, 96, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 96, 96, 3)      │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 3, 3, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,897,168 (60.64 MB)

 Trainable params: 15,897,168 (60.64 MB)

 Non-trainable params: 0 (0.00 B)

### Entrenamiento

In [8]:
model3.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

model3.fit(x_train, y_train, epochs=30, batch_size=32)

Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 237s 118ms/step - accuracy: 0.0974 - loss: 2.3033
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 222s 118ms/step - accuracy: 0.0986 - loss: 2.3028
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 222s 118ms/step - accuracy: 0.1003 - loss: 2.3028
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 223s 119ms/step - accuracy: 0.1029 - loss: 2.3027
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 224s 119ms/step - accuracy: 0.0993 - loss: 2.3027
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 223s 119ms/step - accuracy: 0.0995 - loss: 2.3027
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 222s 119ms/step - accuracy: 0.1018 - loss: 2.3027
Epoch 8/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 222s 119ms/step - accuracy: 0.1007 - loss: 2.3028
Epoch 9/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 223s 119ms/step - accuracy: 0.1001 - loss: 2.3027
Epoch 10/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 223s 119ms/step - accuracy: 0.0978 - loss: 2.3028
Epoch 11/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 223s 119ms/step - accuracy: 0.0960 -

## Modelo 4: VGG16 con entrenamiento previo (ImageNet)

In [9]:
# Cargar VGG16 preentrenado
vgg_base_pretrained = VGG16(
    weights='imagenet',     # preentrenado
    include_top=False,
    input_shape=(96, 96, 3)
)

# Congelar todas las capas del modelo base
vgg_base_pretrained.trainable = False

# Construir modelo final
model4 = Sequential()

# --- BLOQUE DE ADAPTACIÓN (Nuevo) ---
model4.add(Input(shape=(28, 28, 1)))       # Entrada: imágenes pequeñas (28x28)
model4.add(Resizing(96, 96))               # El modelo las agranda a 96x96
model4.add(Conv2D(3, (1, 1), padding='same')) # Convierte 1 canal a 3 canales (RGB)

# --- BLOQUE VGG Y CLASIFICACIÓN ---
model4.add(vgg_base_pretrained)
model4.add(Flatten())
model4.add(Dense(256, activation='relu'))
model4.add(Dropout(0.5))
model4.add(Dense(10, activation='softmax'))

model4.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing_1 (Resizing)           │ (None, 96, 96, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 96, 96, 3)      │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 3, 3, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,897,168 (60.64 MB)

 Trainable params: 1,182,480 (4.51 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

### Entrenamiento

In [10]:
model4.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

model4.fit(x_train, y_train, epochs=15, batch_size=32)

Epoch 1/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 142s 73ms/step - accuracy: 0.7687 - loss: 0.6551
Epoch 2/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 140s 75ms/step - accuracy: 0.8680 - loss: 0.3601
Epoch 3/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 140s 75ms/step - accuracy: 0.8810 - loss: 0.3218
Epoch 4/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 140s 75ms/step - accuracy: 0.8842 - loss: 0.3138
Epoch 5/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 140s 75ms/step - accuracy: 0.8917 - loss: 0.2895
Epoch 6/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 140s 75ms/step - accuracy: 0.8974 - loss: 0.2772
Epoch 7/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 140s 75ms/step - accuracy: 0.9034 - loss: 0.2603
Epoch 8/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 140s 75ms/step - accuracy: 0.9037 - loss: 0.2613
Epoch 9/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 140s 75ms/step - accuracy: 0.9071 - loss: 0.2507
Epoch 10/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 140s 75ms/step - accuracy: 0.9103 - loss: 0.2408
Epoch 11/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 140s 75ms/step - accuracy: 0.9112 - loss: 0.23

# 3. Evaluar modelo

In [19]:
test_loss1, test_acc1 = model1.evaluate(x_test, y_test)
test_loss2, test_acc2 = model2.evaluate(x_test, y_test)
test_loss3, test_acc3 = model3.evaluate(x_test, y_test)
test_loss4, test_acc4 = model4.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9100 - loss: 0.2583
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9287 - loss: 0.2205
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 37ms/step - accuracy: 0.0969 - loss: 2.3026
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 39ms/step - accuracy: 0.9129 - loss: 0.2645
